In [ ]:
import math
import numpy as np
import torch
from torch import nn
from d2l import torch as d2l

# 使用以下三阶多项式来生成训练和测试数据的标签：

max_degree = 20  # 多项式的最大阶数
n_train, n_test = 100, 100  # 训练和测试数据集大小
true_w = np.zeros(max_degree)  # 分配大量的空间
true_w[0:4] = np.array([5, 1.2, -3.4, 5.6])  # 前面4个是真实w，后面的全是0是噪音项

features = np.random.normal(size=(n_train + n_test, 1))  # 200,1
np.random.shuffle(features)
poly_features = np.power(
    features, np.arange(max_degree).reshape(1, -1)
)  # power(x,y)计算x的y次方  poly_features 200*20
for i in range(max_degree):
    poly_features[:, i] /= math.gamma(i + 1)  # gamma(n)=(n-1)!  阶乘
# labels的维度:(n_train+n_test,)
labels = np.dot(poly_features, true_w)  # 乘上真实的偏置  每一行20个算点积出来一个值，所以200*20变成 200
labels += np.random.normal(scale=0.1, size=labels.shape)  # 搞点噪音
# NumPyndarray转换为tensor
true_w, features, poly_features, labels = [
    torch.tensor(x, dtype=torch.float32)
    for x in [true_w, features, poly_features, labels]
]


def evaluate_loss(net, data_iter, loss):
    """评估给定数据集上模型的损失"""
    metric = d2l.Accumulator(2)  # 损失的总和,样本数量
    for X, y in data_iter:
        out = net(X)
        y = y.reshape(out.shape)
        l = loss(out, y)
        metric.add(l.sum(), l.numel())
    return metric[0] / metric[1]


def train(train_features, test_features, train_labels, test_labels, num_epochs=400):
    """训练函数"""
    loss = nn.MSELoss(reduction="none")  # 均方损失函数 (y_hat-y)^2
    # 如果 reduction = ‘none’，直接返回向量形式的 loss
    # 如果 reduction ≠ ‘none’，那么 loss 返回的是标量
    # 如果 reduction=‘mean’，返回 loss.mean(); 注意：默认情况下， reduction=‘mean’
    # 如果 reduction=‘sum’，返回 loss.sum();
    input_shape = train_features.shape[-1]  # 取最后一个元素
    # 不设置偏置，因为我们已经在多项式特征中实现了它 (这里的多项式特征已经是多次项的值了，所以可以用线性回归拟合)
    net = nn.Sequential(nn.Linear(input_shape, 1, bias=False))
    # 多项式回归 https://www.bilibili.com/video/BV164411b7dx?p=22
    batch_size = min(10, train_labels.shape[0])
    train_iter = d2l.load_array(
        (train_features, train_labels.reshape(-1, 1)), batch_size
    )
    test_iter = d2l.load_array(
        (test_features, test_labels.reshape(-1, 1)), batch_size, is_train=False
    )
    trainer = torch.optim.SGD(net.parameters(), lr=0.001)
    animator = d2l.Animator(
        xlabel="epoch",
        ylabel="loss",
        yscale="log",
        xlim=[1, num_epochs],
        ylim=[1e-3, 1e2],
        legend=["train", "test"],
    )
    for epoch in range(num_epochs):
        d2l.train_epoch_ch3(net, train_iter, loss, trainer)
        if epoch == 0 or (epoch + 1) % 20 == 0:
            animator.add(
                epoch + 1,
                (
                    evaluate_loss(net, train_iter, loss),
                    evaluate_loss(net, test_iter, loss),
                ),
            )
    print("weight:", net[0].weight.data.numpy())


In [ ]:
# 正态
# 从多项式特征中选择前4个维度，即1,x,x^2/2!,x^3/3!
train(
    poly_features[:n_train, :4],  # 取前100行的前四列
    poly_features[n_train:, :4],  # 取后100行的后四列
    labels[:n_train],  # 0-100个
    labels[n_train:],  # 100-200个
)

In [ ]:
# 欠拟合 :损失根本就没降，根本没训练好模型，数据（特征）没给全，模型再好也没办法训练出来
# 从多项式特征中选择前2个维度，即1和x
train(
    poly_features[:n_train, :2],
    poly_features[n_train:, :2],
    labels[:n_train],
    labels[n_train:],
)



In [ ]:
# 过拟合 噪音都拟合了
# 从多项式特征中选取所有维度
train(poly_features[:n_train, :], poly_features[n_train:, :],
      labels[:n_train], labels[n_train:], num_epochs=1500)

通过调整拟合多项式的阶数来限制模型的容量，限制特征的数量是缓解过拟合的一种常用技术